In [2]:
# Music Grouping with Spotify API
### Goal
###To fetch song metadata and audio features using Spotify API, and group songs based on similarities using clustering.

from dotenv import load_dotenv
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

# Load credentials from .env
load_dotenv()
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Token endpoint URL
TOKEN_URL = "https://accounts.spotify.com/api/token"

# Function to fetch access token
def get_access_token(client_id, client_secret):
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(TOKEN_URL, headers=headers, data=data)
    if response.status_code == 200:
        token_info = response.json()
        access_token = token_info['access_token']
        print(f"Access token: {access_token}")
        return access_token
    else:
        print(f"Failed to fetch access token. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return None

# Fetch access token
access_token = get_access_token(client_id, client_secret)

# Initialize Spotipy with the token
if access_token:
    sp = spotipy.Spotify(auth=access_token)
else:
    print("Spotify API authentication failed. Exiting.")
    exit()

# Test Spotify API by fetching categories
def test_spotify_api():
    try:
        categories = sp.categories(limit=5)
        print("Fetched categories:")
        for category in categories['categories']['items']:
            print(f"- {category['name']} (ID: {category['id']})")
    except Exception as e:
        print(f"Error while testing Spotify API: {e}")

# Call test function
test_spotify_api()


Access token: BQANPqsNvzIz9kTp78BKYUQ4GOD11GaSwYqqv2sIClR8CuRVNK872R4YHXzkrM46j8GSmt-qu7y94xVlXVFxOiFUmrxZtKDyAfN1RrHS3EaaS02Q5r4
Fetched categories:
- 2024 in Music (ID: 0JQ5DAqbMKFC7do0jUgBzi)
- Made For You (ID: 0JQ5DAt0tbjZptfcdMSKl3)
- New Releases (ID: 0JQ5DAqbMKFGaKcChsSgUO)
- Happy Holidays (ID: 0JQ5DAqbMKFDKyRxRDLIbk)
- Hip-Hop (ID: 0JQ5DAqbMKFQ00XGBls6ym)


In [3]:
# Function to fetch tracks from a Spotify playlist
def get_playlist_tracks(playlist_id, access_token, limit=50):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "limit": limit
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        results = response.json()
        tracks = [
            {
                'name': item['track']['name'],
                'artist': item['track']['artists'][0]['name'],
                'id': item['track']['id']
            }
            for item in results['items']
        ]
        return tracks
    else:
        raise Exception(f"Failed to fetch playlist tracks. Status code: {response.status_code}, Response: {response.text}")

# Replace with your actual playlist ID 
playlist_id = "67QRExTNn4kytNrr3ugJ3B"

# Fetch tracks and handle errors
try:
    tracks = get_playlist_tracks(playlist_id, access_token)
    print(f"Fetched {len(tracks)} tracks.")
    for track in tracks:
        print(f"- {track['name']} by {track['artist']}")
except Exception as e:
    print(f"Error fetching playlist: {e}")


Fetched 50 tracks.
- APT. by ROSÉ
- That’s So True by Gracie Abrams
- luther (with sza) by Kendrick Lamar
- BIRDS OF A FEATHER by Billie Eilish
- Taste by Sabrina Carpenter
- Die With A Smile by Lady Gaga
- Good Luck, Babe! by Chappell Roan
- Sailor Song by Gigi Perez
- toxic till the end by ROSÉ
- WILDFLOWER by Billie Eilish
- Please Please Please by Sabrina Carpenter
- HOT TO GO! by Chappell Roan
- Timeless (with Playboi Carti) by The Weeknd
- A Bar Song (Tipsy) by Shaboozey
- Guess featuring billie eilish by Charli xcx
- Diet Pepsi by Addison Rae
- Moonlit Floor (Kiss Me) by LISA
- EL CLúB by Bad Bunny
- I Love You, I'm Sorry by Gracie Abrams
- MILLION DOLLAR BABY by Tommy Richman
- Sticky (feat. GloRilla, Sexyy Red & Lil Wayne) by Tyler, The Creator
- Espresso by Sabrina Carpenter
- 2 hands by Tate McRae
- I Had Some Help (Feat. Morgan Wallen) by Post Malone
- Too Sweet by Hozier
- Mantra by JENNIE
- No One Noticed by The Marías
- Messy by Lola Young
- NEW DROP by Don Toliver
- Qué

In [5]:
print(track_ids)

['5vNRhkKd0yEAg8suGBpjeY', '7ne4VBA60CxGM75vw0EYad', '45J4avUb9Ni0bnETYaYFVJ', '6dOtVTDdiauQNBQEDOtlAB', '1d7Ptw3qYcfpdLNL5REhtJ', '2plbrEY59IikOBgBGLjaoe', '0WbMK4wrZ1wFSty9F7FCgu', '2262bWmqomIaJXwCRHr13j', '1z5ebC9238uGoBgzYyvGpQ', '3QaPy1KgI7nu9FJEQUgn6h', '5N3hjp1WNayUPZrA8kJmJP', '4xdBrk0nFZaP54vvZj0yx7', '1Es7AUAhQvapIcoh3qMKDL', '2FQrifJ1N335Ljm3TjTVVf', '3WOhcATHxK2SLNeP5W3v1v', '6MzofobZt2dm0Kf1hTThFz', '5G345YEhvleYxQLfYUlEFv', '3zOgFY5kpQ6p2cwPIquZks', '51rfRCiUSvxXlCSCfIztBy', '5AJ9hqTS2wcFQCELCFRO7A', '3tFed7YsjGnIfxeLEQwx3R', '2qSkIjg1o9h3YT9RAgYN75', '1f18HzFpegqvH4ibGJyeMJ', '7221xIgOnuakPdLqT0F3nP', '4IadxL6BUymXlh8RCJJu7T', '2CspwnypzT7rcWI9RfsoSb', '3siwsiaEoU4Kuuc9WKMUy5', '3SKH53SPQbEnZR4cJPVaz2', '2c7z5oRcPBbqRaBY2mLWcf', '0l0vcZMU7AOeQmUIREoI2U', '3Vr3zh0r7ALn8VLqCiRR10', '46kspZSY3aKmwQe7O77fCC', '6tNQ70jh4OwmPGpYy6R2o9', '7hR22TOX3RorxJPcsz5Wbo', '1cOboCuWYI2osTOfolMRS6', '19KlZwqlT3fguP2BeHF1Q1', '4lriIG2vNqwDWzOj2I9rtj', '0nj9Bq5sHDiTxSHunhgkFb', '2uqYupMHAN

In [4]:
#Fetch Audio features for tracks
def get_audio_features(track_ids, access_token):
    url = "https://api.spotify.com/v1/audio-features"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"ids": ",".join(track_ids)}  # Join track IDs as a comma-separated string

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['audio_features']
    else:
        raise Exception(f"Failed to fetch audio features. Status code: {response.status_code}, Response: {response.text}")

# Get track IDs
track_ids = [track['id'] for track in tracks]

# Fetch audio features
audio_features = get_audio_features(track_ids, access_token)
print(f"Fetched audio features for {len(audio_features)} tracks.")

Exception: Failed to fetch audio features. Status code: 403, Response: {
  "error" : {
    "status" : 403
  }
}